In [1]:
import asyncio
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm_asyncio
from tqdm import tqdm
import pandas as pd

client = AsyncOpenAI(base_url='http://0.0.0.0:8000/v1',api_key='nvm')
model_name="gptoss120b/"
async def generate_async(prompt, idx):
    try:
        response = await client.responses.create(
            model=model_name,  # or your preferred model
            input=[
                {"role": "user", "content": prompt}
            ],
            temperature=1.0,
            top_p=1.0
        )
        
        return idx, response.output[1].content[0].text
    except Exception as e:
        return idx, f"Error: {str(e)}"


In [2]:
developer_prompt="""You are an expert-level examiner creating extremely difficult MCQ questions about {topic}.

CRITICAL RULES:
1. Generate ONLY ONE question per response (not multiple)
2. Topic must be strictly: {topic}
3. Make questions genuinely hard - trick 50%+ of experts
4. keep in mind that it cannot include any numerical style seating arrangement e.g. how many permutations such arragement possible etc.
5. Return ONLY valid JSON (no other text)

FORMAT (must be exact):
{{
    "topic": "<Topic of the Question>",
    "question": "<full question text>",
    "choices": [
        "A) <choice A text>",
        "B) <choice B text>",
        "C) <choice C text>",
        "D) <choice D text>"
    ],
    "answer": "<correct choice letter only>",
    "explanation": "brief explanation within 100 words for why the answer is correct"
}}"""

In [ ]:
async def main():
    topics = [
    "Logical Reasoning: Syllogisms",
    "Puzzles: Seating Arrangements (Linear, Circular)",
    "AI",
    "NLP"]

    tasks = [generate_async(developer_prompt.format(topic=topics[1]), idx) for idx in range(1000)]

    results = [None] * len(tasks)

    for future in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
        idx, output = await future
        results[idx] = output

    return results


results = await main()

final_output=[]
for i, r in enumerate(results):
    print(f"\n[{i}] {r}")
    final_output.append(eval(r))
    pd.DataFrame({'answer':final_output}).to_csv('Puzzles.csv',index=False)


  1%|          | 9/1000 [01:28<56:54,  3.45s/it]   

In [1]:
import os
hf_cache = "/root/.cache/huggingface"
print("HuggingFace cache contents:")
for item in os.listdir(hf_cache):
    print(f"  {item}")
    if os.path.isdir(os.path.join(hf_cache, item)):
        subpath = os.path.join(hf_cache, item)
        for subitem in os.listdir(subpath)[:5]:
            print(f"    └─ {subitem}")

HuggingFace cache contents:
  .locks
    └─ models--microsoft--Phi-4-mini-instruct
    └─ models--Unsloth--Llama-3.1-8B-Instruct
    └─ models--mistralai--Mistral-7B-Instruct-v0.3
    └─ models--unsloth--gpt-oss-20b-BF16
    └─ models--google--gemma-3-12b-it
  xet
    └─ https___cas_serv-tGqkUaZf_CBPHQ6h
    └─ logs
  models--microsoft--Phi-4-mini-instruct
    └─ snapshots
    └─ refs
    └─ blobs
  models--Unsloth--Llama-3.1-8B-Instruct
    └─ snapshots
    └─ refs
    └─ blobs
  models--mistralai--Mistral-7B-Instruct-v0.3
    └─ snapshots
    └─ refs
    └─ blobs
  models--unsloth--gpt-oss-20b-BF16
    └─ snapshots
    └─ refs
    └─ blobs
  models--google--gemma-3-12b-it
    └─ snapshots
    └─ refs
    └─ blobs
  models--Qwen--Qwen2.5-14B-Instruct
    └─ snapshots
    └─ refs
    └─ blobs
  models--Qwen--Qwen3-4B
    └─ snapshots
    └─ refs
    └─ blobs


In [2]:
import os
snap_path = "/root/.cache/huggingface/models--Qwen--Qwen2.5-14B-Instruct/snapshots"
snapshots = os.listdir(snap_path)
print(f"Available snapshots: {snapshots}")
if snapshots:
    snap_hash = snapshots[0]
    full_path = os.path.join(snap_path, snap_hash)
    print(f"Full path: {full_path}")
    print(f"Contents: {os.listdir(full_path)[:10]}")  # First 10 items

Available snapshots: ['cf98f3b3bbb457ad9e2bb7baf9a0125b6b88caa8']
Full path: /root/.cache/huggingface/models--Qwen--Qwen2.5-14B-Instruct/snapshots/cf98f3b3bbb457ad9e2bb7baf9a0125b6b88caa8
Contents: ['model-00008-of-00008.safetensors', 'model-00006-of-00008.safetensors', 'generation_config.json', 'model-00005-of-00008.safetensors', 'README.md', 'config.json', 'model.safetensors.index.json', 'vocab.json', 'model-00003-of-00008.safetensors', 'LICENSE']


In [4]:
!find .. -name "questions_training.json" -type f


../AAIPL/questions_training.json


In [5]:
import os
import glob

# Check multiple possible locations
locations_to_check = [
    "../AAIPL/questions_training.json",
    "../../questions_training.json",
    "/workspace/AAIPL/questions_training.json",
    "questions_training.json",
    "../questions_training.json",
]

print("Checking locations:")
for loc in locations_to_check:
    exists = os.path.exists(loc)
    print(f"  {loc}: {'✓ FOUND' if exists else '✗ Not found'}")

# Also search for the file
print("\nSearching for questions_training.json:")
result = glob.glob("/workspace/**/questions_training.json", recursive=True)
if result:
    for path in result:
        print(f"  Found: {path}")
else:
    print("  No results found")

# Print current working directory
print(f"\nCurrent directory: {os.getcwd()}")
print(f"Files in current directory: {os.listdir('.')}")


Checking locations:
  ../AAIPL/questions_training.json: ✓ FOUND
  ../../questions_training.json: ✗ Not found
  /workspace/AAIPL/questions_training.json: ✓ FOUND
  questions_training.json: ✓ FOUND
  ../questions_training.json: ✗ Not found

Searching for questions_training.json:
  Found: /workspace/AAIPL/questions_training.json

Current directory: /workspace/AAIPL
Files in current directory: ['README.ipynb', 'agen.yaml', 'qgen.yaml', 'utils', 'assets', '.dockerignore', '.ipynb_checkpoints', 'tutorial_config.yaml', 'tutorial.ipynb', 'agents', 'hf_models', 'tokenizers.py', 'Logical_Reasoning.csv', 'questions_training.json', 'gptoss120b', 'SETUP.md', 'data_gen_v2-family.py', 'EXECUTION_GUIDE.md', 'scripts', 'data_gen_v2.ipynb', 'requirements.txt', 'IMPLEMENTATION_PLAN.md', 'unsloth_compiled_cache', 'Mixed_Series.csv', 'data_gen_v2-series.py', 'Family_Tree.csv', 'Puzzles.csv', 'data_gen_v2.py', 'git.sh']


In [7]:
import json
data = json.load(open('questions_training.json'))
print('First item:')
print(json.dumps(data[0], indent=2))


First item:
{
  "answer": "{'topic': 'Logical Reasoning: Syllogisms', 'question': 'Consider the following statements:\\n1. No architect is a surgeon.\\n2. Some engineer is an architect.\\n3. All surgeons are poets.\\n4. Some poets are not engineers.\\n5. No engineer is a detective.\\n6. Every detective is a botanist.\\n7. Some botanist is an architect.\\nWhich of the following conclusions necessarily follows from the above statements?', 'choices': ['A) Some poets are engineers.', 'B) Some architects are botanists.', 'C) No surgeons are detectives.', 'D) All detectives are not architects.'], 'answer': 'B', 'explanation': \"Statement 7 asserts that some botanists are architects, which directly translates to 'some architects are botanists'. None of the other options are guaranteed by the given premises.\"}"
}


In [8]:
import json
import ast

# Load the malformed data
print("Loading questions_training.json...")
with open('questions_training.json', 'r') as f:
    data = json.load(f)

print(f"Loaded {len(data)} items")

# Fix the structure
fixed_data = []
errors = 0

for idx, item in enumerate(data):
    try:
        # Parse the stringified dict in the "answer" field
        if isinstance(item.get('answer'), str):
            # Try to parse as Python literal
            question_data = ast.literal_eval(item['answer'])
        else:
            question_data = item['answer']

        fixed_data.append(question_data)
    except Exception as e:
        errors += 1
        if errors <= 3:  # Print first 3 errors
            print(f"Error at index {idx}: {e}")
        continue

# Save the fixed data
print(f"\nFixed {len(fixed_data)} questions ({errors} errors)")
with open('questions_training_fixed.json', 'w') as f:
    json.dump(fixed_data, f)

print("Saved to questions_training_fixed.json")

# Verify the first item
if fixed_data:
    print("\nFirst fixed item:")
    print(json.dumps(fixed_data[0], indent=2))


Loading questions_training.json...
Loaded 3450 items

Fixed 3450 questions (0 errors)
Saved to questions_training_fixed.json

First fixed item:
{
  "topic": "Logical Reasoning: Syllogisms",
  "question": "Consider the following statements:\n1. No architect is a surgeon.\n2. Some engineer is an architect.\n3. All surgeons are poets.\n4. Some poets are not engineers.\n5. No engineer is a detective.\n6. Every detective is a botanist.\n7. Some botanist is an architect.\nWhich of the following conclusions necessarily follows from the above statements?",
  "choices": [
    "A) Some poets are engineers.",
    "B) Some architects are botanists.",
    "C) No surgeons are detectives.",
    "D) All detectives are not architects."
  ],
  "answer": "B",
  "explanation": "Statement 7 asserts that some botanists are architects, which directly translates to 'some architects are botanists'. None of the other options are guaranteed by the given premises."
}
